In [1]:
import numpy as np
import json
import pandas as pd

In [2]:
dataDir = "./data/openface"

In [3]:
def readOpenFaceFile(filename) -> [dict]:
  lines = []
  with open(f"{dataDir}/{filename}", "r") as f:
      for l in f:
        detectionStr = l.split(",", maxsplit=1)[1].rstrip().strip("\"")
        detectionStr = detectionStr.replace("\'", "\"")
        lines.append(json.loads(detectionStr))
  return lines

In [4]:
def getNumOfNoDetections(frames: [dict]) -> int:
  return len([f for f in frames if f["intensity"] == "-1" or f["intensity"]["AU01"] == "-"])

In [5]:
def getNumOfZeroIntensity(frames: [dict]) -> int:
  return len([f for f in frames if f["intensity"] != "-1" and f["intensity"]["AU06"] == 0 and f["intensity"]["AU12"] == 0])

In [6]:
videos = ["bright", "dark", "outside", "moving_background", "hololens_left", "hololens_right"]
bitrates = ["750", "1500"]
methods = ["none", "face", "fes"]

In [7]:
def print_stats(data, name):
    print("Avg {} per method".format(name))
    a = data.reshape(len(videos)*len(bitrates), len(methods))
    avgs = np.mean(a, axis=0)
    print(pd.DataFrame(avgs, index=methods))

    print("\nAvg {} per video".format(name))
    a = data.reshape(len(videos), len(bitrates)*len(methods))
    avgs = np.mean(a, axis=1)
    print(pd.DataFrame(avgs, index=videos))

    print("\nAvg {} per bitrate".format(name))
    a = data.swapaxes(1, 2).reshape(len(videos)*len(methods), len(bitrates))
    avgs = np.mean(a, axis=0)
    print(pd.DataFrame(avgs, index=bitrates))

    print("\nAvg {} method and bitrate".format(name))
    avgs = np.mean(data, axis=0)
    print(pd.DataFrame(avgs, index=bitrates, columns=methods))

    print("\nAvg {} per video and bitrate".format(name))
    avgs = np.mean(data, axis=2)
    print(pd.DataFrame(avgs, index=videos, columns=bitrates))

    print("\nAvg {} per video and method".format(name))
    avgs = np.mean(data, axis=1)
    print(pd.DataFrame(avgs, index=videos, columns=methods))


In [8]:
valuesDetect = []

for v in videos:
    for b in bitrates:
        for m in methods:
            frames = readOpenFaceFile(f"{v}_{b}_{m}.csv")
            noDetecs = getNumOfNoDetections(frames)
            zeroItensity = getNumOfZeroIntensity(frames)
            ratio = noDetecs/len(frames)
            valuesDetect.append(ratio)
            # print(f"{v}_{b}_{m}: {noDetecs}/{len(frames)} = {ratio:.3f}")

valuesDetect = np.array(valuesDetect).reshape(len(videos),len(bitrates),len(methods))

In [9]:
print_stats(valuesDetect, "no detections")

Avg no detections per method
             0
none  0.230311
face  0.224007
fes   0.223008

Avg no detections per video
                          0
bright             0.224322
dark               0.267775
outside            0.202307
moving_background  0.200369
hololens_left      0.293542
hololens_right     0.166337

Avg no detections per bitrate
             0
750   0.209835
1500  0.241716

Avg no detections method and bitrate
          none      face       fes
750   0.215897  0.201522  0.212087
1500  0.244725  0.246493  0.233929

Avg no detections per video and bitrate
                        750      1500
bright             0.206069  0.242576
dark               0.246365  0.289186
outside            0.188786  0.215828
moving_background  0.179382  0.221356
hololens_left      0.288563  0.298520
hololens_right     0.149847  0.182827

Avg no detections per video and method
                       none      face       fes
bright             0.246368  0.209236  0.217363
dark               0.276